In [1]:
import torch
import os
import sys
from mqbench.convert_deploy import convert_deploy
from mqbench.prepare_by_platform import prepare_by_platform, BackendType
from mqbench.utils.state import enable_calibration, enable_quantization

In [2]:
import mqbench
from mqbench.convert_deploy import convert_deploy

In [3]:
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [4]:
import tqdm

In [ ]:
resnet18 = models.resnet18(pretrained=True)

In [ ]:
def test(model, device, test_loader):
    old_training_state = model.training
    model.eval()
    test_loss = 0
    correct = 0
    lossLayer = torch.nn.CrossEntropyLoss(reduction='sum')
    for data, target in tqdm.tqdm(test_loader):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        test_loss += lossLayer(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    model.train(old_training_state)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {:.3f}%\n'.format(
        test_loss, 100. * correct / len(test_loader.dataset)
    ))

In [ ]:
# change valdir to your imagenet dataset validation directory
valdir = 'yourdisk/ImageNetDataset/val/'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),batch_size=128, shuffle=False, num_workers=24, pin_memory=True)

In [ ]:
device = torch.device('cuda')
resnet18.to(device)
# 69.758, consistent with reported at https://pytorch.org/vision/stable/models.html
test(resnet18, device, val_loader)

In [ ]:
model = resnet18.cpu().train()
model_mqbench = prepare_by_platform(model, BackendType.OPENVINO)

In [ ]:
enable_calibration(model_mqbench)

model_mqbench.to(device)
model_mqbench.eval()
# calibration loop
for data, target in tqdm.tqdm(val_loader):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            model_mqbench(data)

In [ ]:
enable_quantization(model_mqbench)
model_mqbench.train()
# QAT loop ...

In [ ]:
test(model_mqbench, device, val_loader)

In [ ]:
input_shape={'data': [32, 3, 224, 224]}
convert_deploy(model_mqbench.eval(), BackendType.OPENVINO, input_shape)

# test openvino

In [ ]:
!mo --input_model mqbench_qmodel_deploy_model.onnx 

In [ ]:
import logging as log
import numpy as np
from openvino.inference_engine import IECore

In [ ]:
log.basicConfig(format="[ %(levelname)s ] %(message)s", level=log.ERROR, stream=sys.stdout)
log.info("Creating Inference Engine...")
ie = IECore()

In [ ]:
# Read IR
log.info("Loading network")
net = ie.read_network("./mqbench_qmodel_deploy_model.xml")

img_info_input_blob = None
feed_dict = {}
input_blob = "input"
for blob_name in net.input_info:
    if len(net.input_info[blob_name].input_data.shape) == 4:
        input_blob = blob_name
    elif len(net.input_info[blob_name].input_data.shape) == 2:
        img_info_input_blob = blob_name
    else:
        raise RuntimeError("Unsupported {}D input layer '{}'. Only 2D and 4D input layers are supported"
                           .format(len(net.input_info[blob_name].input_data.shape), blob_name))

log.info("Loading IR to the plugin...")

In [ ]:
exec_net = ie.load_network(network=net, num_requests=4, device_name="CPU")
n, c, h, w = net.input_info[input_blob].input_data.shape
if img_info_input_blob:
    feed_dict[img_info_input_blob] = [h, w, 1]

In [ ]:
n, c, h, w

In [ ]:
import torch.nn.functional as F

In [ ]:
val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ])),batch_size=32, shuffle=False, num_workers=24, pin_memory=True, drop_last=True)

In [ ]:
def test_openvino(exec_net, test_loader, model):
    
    global feed_dict
    global input_blob
    cur_request_id = 0
    
    test_loss = 0
    correct = 0
    
    def infer(img):
        cur_request_id = 0
        feed_dict[input_blob] = img
        exec_net.start_async(request_id=cur_request_id, inputs=feed_dict)
        while exec_net.requests[cur_request_id].wait(-1) != 0:
            pass
        outs = exec_net.requests[cur_request_id].output_blobs
        k = tuple(outs.keys())[0]
        return outs[k].buffer
    
    lossLayer = torch.nn.CrossEntropyLoss(reduction='sum')
    device = torch.device('cuda')
    old_training_state = model.training
    model.to(device)
    model.eval()
    cosine_distance = 0
    for data, target in tqdm.tqdm(test_loader):
        output = infer(data.numpy())
        output = torch.as_tensor(output)
        with torch.no_grad():
            output_torch = model(data.to(device)).cpu()
        cosine_distance += (F.cosine_similarity(output, output_torch)).sum()
        test_loss += lossLayer(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    cosine_distance /= len(test_loader.dataset)
    test_loss /= len(test_loader.dataset)
    model.train(old_training_state)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {:.3f%}, cosine_distance:{:.4f}\n'.format(
        test_loss, 100. * correct / len(test_loader.dataset), cosine_distance
    ))
    return cosine_distance

In [ ]:
test_openvino(exec_net, val_loader, model_mqbench)

In [ ]:
torch.onnx.export(resnet18.cpu(), torch.rand(1,3,224,224), 'resnet18.onnx', do_constant_folding=True,
                      input_names=["input"], output_names=["output"], opset_version=11)

In [ ]:
!mo --input_model resnet18.onnx 

In [ ]:
input_shape={'data': [1, 3, 224, 224]}
convert_deploy(model_mqbench.cpu().eval(), BackendType.OPENVINO, input_shape)

In [ ]:
!mo --input_model resnet18.onnx 

In [ ]:
!mo --input_model mqbench_qmodel_deploy_model.onnx

In [ ]:
!benchmark_app -m resnet18.xml -nstreams 1

In [ ]:
!benchmark_app -m mqbench_qmodel_deploy_model.xml  -nstreams 1